overpy ist eine Python-Bibliothek, die es ermöglicht, Daten von der Overpass API (eine Schnittstelle für OpenStreetMap-Daten) abzufragen und zu verarbeiten. Die Overpass API ermöglicht es, Wanderwege aus dem OpenStreetMap-Projekt (OSM) abzurufen.

Die Abrage sucht nach Wanderrouten in einem Giebiet welche mit spezifischen Signalisationen ausgeschildert sind. 

Falls das der Name nicht vorhanden ist, aber die Bezeichnungen von und bis existieren, wird der Name zusammengesetzt.



In [ ]:
import overpy
import pandas as pd

# Initialisiere die Overpass API mit einer benutzerdefinierten URL
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

# Overpass Query für Wanderwege innerhalb der Schweiz. Mit Center erhalten wir die Koordinaten in der Mitte eines Wnaderweges
query = """
[out:json];
relation
["route"="hiking"]
/*["name"]*/
["name"!~"fixme", i]
["network"="lwn"]
["osmc:symbol"~"yellow::yellow_diamond|red:white:red_bar|yellow:white:yellow_diamond|blue:white:blue_bar"]
/*(id: 1432463)*/
(45.8899, 6.0872, 47.8085, 10.4921);
out center tags;
"""

# Führe die Anfrage aus
result = api.query(query)

# Liste zum Speichern der extrahierten Informationen
list = []

# Iteration über alle relations
for relation in result.relations:
    
    # Extrahiere die relevanten Daten

    org_name = relation.tags.get('name')
    fix_name = ""
    org_to = relation.tags.get('to')
    org_from = relation.tags.get('from')
    
    # Center ist ein Tupel mit langitute und longitude, wir möchten nur einen wert
    lat = getattr(relation, 'center_lat')
    lon = getattr(relation, 'center_lon')
    
    # Wenn der orginal Name nicht vorhanden ist setze ihn aus from und to zusammen
    if not org_name and org_from and org_to :
        fix_name = f"{org_from} - {org_to}"
    else:
        fix_name = org_name

    # Erstelle ein Dictionary um die Attribute als Tupel zu speichern
    if fix_name and lat > 0 and lon > 0: 
        dict = {
        'id': relation.id,
        'name': fix_name,
        'lat': lat,
        'lon': lon
        }

        # Jedes Tupel wird nun in der Liste als neue Zeile gespeichert
        list.append(dict)

# Wenn alle Daten verarbeitet wurden, erstelle das DataFrame
df = pd.DataFrame(list)

# Zeige das DataFrame an
print(df)


sqlalchemy dient dazu, SQL-Datenbanken mit Python-Code zu verbinden und erleichtert das Arbeiten mit relationalen Datenbanken. sqlalchemy bietet einen direkten Zugriff auf SQL-Datenbanken, was Flexibilität ermöglicht.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Verbindung zur MySQL-Datenbank herstellen (ersetze Platzhalter mit deinen Daten)
user = 'root'        # MySQL-Benutzername
password = ''  # MySQL-Passwort
host = '127.0.0.1'   # Server (bei lokalen Installationen ist das oft 'localhost')
database = 'mydatabase'  # Name der Datenbank, in die du schreiben möchtest

# Erstelle die Verbindungs-Engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# DataFrame in die MySQL-Datenbank-Tabelle schreiben (wenn die Tabelle nicht existiert, wird sie erstellt)
df.to_sql(name='Wanderwege', con=engine, if_exists='replace', index=False)

print("DataFrame erfolgreich in die MySQL-Datenbank geladen!")